non factorizable n (or at least no entries in factordb)
maybe the broken part is n being p or q

In [107]:
n =  27772857409875257529415990911214211975844307184430241451899407838750503024323367895540981606586709985980003435082116995888017731426634845808624796292507989171497629109450825818587383112280639037484593490692935998202437639626747133650990603333094513531505209954273004473567193235535061942991750932725808679249964667090723480397916715320876867803719301313440005075056481203859010490836599717523664197112053206745235908610484907715210436413015546671034478367679465233737115549451849810421017181842615880836253875862101545582922437858358265964489786463923280312860843031914516061327752183283528015684588796400861331354873
e = 16
c = 11303174761894431146735697569489134747234975144162172162401674567273034831391936916397234068346115459134602443963604063679379285919302225719050193590179240191429612072131629779948379821039610415099784351073443218911356328815458050694493726951231241096695626477586428880220528001269746547018741237131741255022371957489462380305100634600499204435763201371188769446054925748151987175656677342779043435047048130599123081581036362712208692748034620245590448762406543804069935873123161582756799517226666835316588896306926659321054276507714414876684738121421124177324568084533020088172040422767194971217814466953837590498718
m = 27772857409875257529415990911214211975844307184430241451899407838750503024323367895540981606586709985980003435082116995884478493648417679670782240786518871242668255973770988737167589482305554697589427755948133831217194514801188587815879226601454549604874476927252882744344317256396808255582550941978968312010016490081931048157003674361093215320945272838509986687284028991543505902180465028764465401474732603047959528463455521587693401684330259430026213913640315266928039931940062910914620141512756390510844994503754868094272416592656824961726496896913664170409316563677657113965998358809997237032168697425106457591932


In [118]:
from cryptolib import long_to_bytes
long_to_bytes(-m % n)

b"Hey, if you are reading this maybe I didn't mess up my code too much. Phew. I really should play more CryptoHack before rushing to code stuff from scratch again. Here's the flag: crypto{m0dul4r_squ4r3_r00t}"

In [88]:
import random
import logging
import sys
from cryptolib import invmod as inverse_modulo, is_prime

_logger = logging.getLogger("tonellishanks")


def legendre_symbol(a: int, p: int, /) -> int:

    assert p % 2 != 0

    return pow(a, (p - 1) >> 1, p)


def _choose_b(p: int, /, *, det=True) -> int:

    assert p > 2
    assert p % 2 != 0

    b = 2
    _attempts = 1

    if det:
        while legendre_symbol(b, p) == 1:
            b += 1
            _attempts += 1
    else:
        while legendre_symbol(b, p) == 1:
            b = random.randrange(2, p)
            _attempts += 1

    assert b < p
    assert legendre_symbol(b, p) == p - 1

    _logger.info("Found b = %d after %d attempts", b, _attempts)

    return b


def _tonelli_shanks_recursive(a: int, k: int, p: int, b: int, b_inverse: int, /):
    """
    Computes a square root of a modulo prime p
    :param a: the number to take the square root of
    :param k: positive integer, such that a^m = 1 (mod p) where m = (p-1)/(2^k)
    :param p: odd prime p modulo which we are working
    :param b: an arbitrary non-square modulo p
    :param b_inverse: the inverse of b modulo p, i.e., b * b_inverse = 1 (mod p)
    :return: one of the square roots of a modulo p (the other can be obtained via negation modulo p)
    """

    assert p > 2
    assert 0 < a < p
    assert k > 0

    m = (p - 1) >> k

    # assumption
    assert pow(a, m, p) == 1

    a_m = 1

    # check that b is indeed a non-square modulo p
    assert legendre_symbol(b, p) == p - 1

    _logger.info("-------- [New round] --------")
    _logger.info("a = %d, m = %d, a^m = 1", a, m)

    while m % 2 == 0 and a_m == 1:

        m >>= 1
        k += 1

        assert m == (p - 1) >> k

        a_m = pow(a, m, p)

        _logger.info(
            "m is even and a^m = 1 => we divide m by 2 and get: m = %d, a^m = %s",
            m,
            "1" if a_m == 1 else "-1"
        )

        # since Z/pZ is a field, there cannot be any roots for 1 apart from 1 and -1
        assert a_m == 1 or a_m == p - 1

    assert a_m == 1 or a_m == p - 1

    if a_m == p - 1:
        # a^m = -1 (mod p)
        _logger.info("m = %d, a^m = -1 => we multiply a^m with a legendre symbol of a non-square b modulo p", m)
        assert k >= 2
        b_power = 1 << (k - 1)
        b_power_half = 1 << (k - 2)
        assert pow(a, m, p) == p - 1
        assert b_power * m == (p - 1) >> 1
        a_next = (a * pow(b, b_power, p)) % p
        _logger.info("(a * b^%d)^m = (a * b^%d)^%d = %d^%d = 1", b_power, b_power, m, a_next, m)
        _logger.info(
            "It follows that a_next := a * b^%d = %d * %d = %d is a square whose root yields a root of a",
            b_power,
            a,
            pow(b, b_power, p),
            a_next
        )
        assert pow(a_next, m, p) == 1
        a_next_root = _tonelli_shanks_recursive(a_next, k, p, b, b_inverse)
        _logger.info("The root of a_next = %d is %d", a_next, a_next_root)
        a_root = a_next_root * pow(b_inverse, b_power_half, p)
        _logger.info("sqrt(a_next)^2 = %d^2 = a_next = a * b^%d = sqrt(a)^2 * b^%d", a_next_root, b_power, b_power)
        _logger.info(
            "=> sqrt(a = %d) = sqrt(a_next) * b^(-%d) = %d * %d = %d",
            a,
            b_power_half,
            a_next_root,
            pow(b_inverse, b_power_half, p),
            a_root
        )
        _logger.info("-------- [Round complete] --------")
        return a_root % p
    assert a_m == 1
    assert m % 2 == 1
    _logger.info("-------- [Round complete] --------")
    # we now handle the case when m is odd
    # this case is easy, a^((m+1)/2) is a square root of a
    return pow(a, (m + 1) >> 1, p)


def tonelli_shanks(a: int, p: int, /, *, deterministic=True) -> int | None:
    """
    Computes a square root of a modulo prime p
    :param a: the number to take the square root of
    :param p: odd prime p modulo which we are working
    :param deterministic: whether to search for the non-square b deterministically
    :return: one of the square roots of a modulo p (the other can be obtained via negation modulo p)
    """
    assert p > 2
    assert 0 < a < p
    # quick Fermat primality test
    assert pow(a, p - 1, p) == 1
    if legendre_symbol(a, p) != 1:
        # a is not not a square modulo p
        return None
    _logger.info("======== [Starting algorithm with a = %d, p = %d] ========", a, p)
    b = _choose_b(p, det=deterministic)
    b_inverse = inverse_modulo(b, p)
    assert b * b_inverse % p == 1
    return _tonelli_shanks_recursive(a, 1, p, b, b_inverse)

In [99]:
tonelli_shanks(tonelli_shanks(tonelli_shanks(tonelli_shanks(c, n), n), n), n)

27772857409875257529415990911214211975844307184430241451899407838750503024323367895540981606586709985980003435082116995884478493648417679670782240786518871242668255973770988737167589482305554697589427755948133831217194514801188587815879226601454549604874476927252882744344317256396808255582550941978968312010016490081931048157003674361093215320945272838509986687284028991543505902180465028764465401474732603047959528463455521587693401684330259430026213913640315266928039931940062910914620141512756390510844994503754868094272416592656824961726496896913664170409316563677657113965998358809997237032168697425106457591932